In [24]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

print(os.listdir("./HousesDataset_ref/"))

['100_bathroom.jpg', '100_bedroom.jpg', '100_frontal.jpg', '100_kitchen.jpg', '101_bathroom.jpg', '101_bedroom.jpg', '101_frontal.jpg', '101_kitchen.jpg', '102_bathroom.jpg', '102_bedroom.jpg', '102_frontal.jpg', '102_kitchen.jpg', '103_bathroom.jpg', '103_bedroom.jpg', '103_frontal.jpg', '103_kitchen.jpg', '104_bathroom.jpg', '104_bedroom.jpg', '104_frontal.jpg', '104_kitchen.jpg', '105_bathroom.jpg', '105_bedroom.jpg', '105_frontal.jpg', '105_kitchen.jpg', '106_bathroom.jpg', '106_bedroom.jpg', '106_frontal.jpg', '106_kitchen.jpg', '107_bathroom.jpg', '107_bedroom.jpg', '107_frontal.jpg', '107_kitchen.jpg', '108_bathroom.jpg', '108_bedroom.jpg', '108_frontal.jpg', '108_kitchen.jpg', '109_bathroom.jpg', '109_bedroom.jpg', '109_frontal.jpg', '109_kitchen.jpg', '10_bathroom.jpg', '10_bedroom.jpg', '10_frontal.jpg', '10_kitchen.jpg', '110_bathroom.jpg', '110_bedroom.jpg', '110_frontal.jpg', '110_kitchen.jpg', '111_bathroom.jpg', '111_bedroom.jpg', '111_frontal.jpg', '111_kitchen.jpg', '1

In [25]:
# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import glob
import cv2
import os
def load_house_attributes(inputPath):
    # initialize the list of column names in the CSV file and then
    # load it using Pandas
    cols = ["bedrooms", "bathrooms", "area", "zipcode", "price"]
    df = pd.read_csv(inputPath, sep=" ", header=None, names=cols)

    # determine (1) the unique zip codes and (2) the number of data
    # points with each zip code
    zipcodes = df["zipcode"].value_counts().keys().tolist()
    counts = df["zipcode"].value_counts().tolist()

    # loop over each of the unique zip codes and their corresponding
    # count
    for (zipcode, count) in zip(zipcodes, counts):
        # the zip code counts for our housing dataset is *extremely*
        # unbalanced (some only having 1 or 2 houses per zip code)
        # so let's sanitize our data by removing any houses with less
        # than 25 houses per zip code
        if count < 25:
            idxs = df[df["zipcode"] == zipcode].index
            df.drop(idxs, inplace=True)

    # return the data frame
    return df

def process_house_attributes(df, train, test):
    # initialize the column names of the continuous data
    continuous = ["bedrooms", "bathrooms", "area"]

    # performin min-max scaling each continuous feature column to
    # the range [0, 1]
    cs = MinMaxScaler()
    trainContinuous = cs.fit_transform(train[continuous])
    testContinuous = cs.transform(test[continuous])

    # one-hot encode the zip code categorical data (by definition of
    # one-hot encoing, all output features are now in the range [0, 1])
    zipBinarizer = LabelBinarizer().fit(df["zipcode"])
    trainCategorical = zipBinarizer.transform(train["zipcode"])
    testCategorical = zipBinarizer.transform(test["zipcode"])

    # construct our training and testing data points by concatenating
    # the categorical features with the continuous features
    trainX = np.hstack([trainCategorical, trainContinuous])
    testX = np.hstack([testCategorical, testContinuous])

# return the concatenated training and testing data
    return (trainX, testX)

def load_house_images(df, inputPath):
    # initialize our images array (i.e., the house images themselves)
    images = []

    # loop over the indexes of the houses
    for i in df.index.values:
        # find the four images for the house and sort the file paths,
        # ensuring the four are always in the *same order*
        basePath = os.path.sep.join([inputPath, "{}_*".format(i + 1)])
        housePaths = sorted(list(glob.glob(basePath)))

        # initialize our list of input images along with the output image
        # after *combining* the four input images
        inputImages = []
        outputImage = np.zeros((64, 64, 3), dtype="uint8")

        # loop over the input house paths
        for housePath in housePaths:
            # load the input image, resize it to be 32 32, and then
            # update the list of input images
            image = cv2.imread(housePath)
            image = cv2.resize(image, (32, 32))
            inputImages.append(image)

        # tile the four input images in the output image such the first
        # image goes in the top-right corner, the second image in the
        # top-left corner, the third image in the bottom-right corner,
        # and the final image in the bottom-left corner
        outputImage[0:32, 0:32] = inputImages[0]
        outputImage[0:32, 32:64] = inputImages[1]
        outputImage[32:64, 32:64] = inputImages[2]
        outputImage[32:64, 0:32] = inputImages[3]

        # add the tiled image to our set of images the network will be
        # trained on
        images.append(outputImage)
        print(np.array(images))
    # return our set of images
    
    return np.array(images)

In [26]:
# import the necessary packages

from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import numpy as np
import os

# construct the path to the input .txt file that contains information
# on each house in the dataset and then load the dataset
print("[INFO] loading house attributes...")
inputPath = os.path.sep.join(["./HousesDataset_ref/","HousesInfo.txt"]) 
df = load_house_attributes(inputPath)
print(df)

[INFO] loading house attributes...
     bedrooms  bathrooms  area  zipcode    price
30          5        3.0  2520    93446   789000
32          3        2.0  1802    93446   365000
39          3        3.0  2146    93446   455000
80          4        2.5  2464    91901   599000
81          2        2.0  1845    91901   529800
..        ...        ...   ...      ...      ...
499         4        4.0  3000    93446  1495000
500         3        2.0  2330    93446   599900
501         3        2.5  1339    93446   344900
502         3        2.0  1472    93446   309995
503         4        4.0  2681    93446   572000

[362 rows x 5 columns]


In [27]:
# load the house images and then scale the pixel intensities to the
# range [0, 1]
print("[INFO] loading house images...")
images = load_house_images(df, "./HousesDataset_ref/")
images = images / 255.0

[INFO] loading house images...
[[[[152 169 178]
   [156 175 183]
   [160 177 186]
   ...
   [ 91 139 170]
   [ 83 135 165]
   [ 80 134 167]]

  [[157 176 184]
   [156 175 183]
   [152 171 179]
   ...
   [ 82 130 162]
   [ 76 131 166]
   [ 69 129 166]]

  [[154 173 181]
   [153 171 182]
   [151 169 180]
   ...
   [ 77 131 166]
   [ 72 127 166]
   [ 64 126 164]]

  ...

  [[149 177 187]
   [192 187 182]
   [190 190 190]
   ...
   [186 202 214]
   [199 215 230]
   [150 166 179]]

  [[212 214 215]
   [215 218 219]
   [223 219 221]
   ...
   [206 219 231]
   [169 183 204]
   [209 224 239]]

  [[215 220 220]
   [218 223 222]
   [214 217 221]
   ...
   [163 164 222]
   [186 209 220]
   [193 212 225]]]]
[[[[152 169 178]
   [156 175 183]
   [160 177 186]
   ...
   [ 91 139 170]
   [ 83 135 165]
   [ 80 134 167]]

  [[157 176 184]
   [156 175 183]
   [152 171 179]
   ...
   [ 82 130 162]
   [ 76 131 166]
   [ 69 129 166]]

  [[154 173 181]
   [153 171 182]
   [151 169 180]
   ...
   [ 77 131 166

In [28]:
# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
split = train_test_split(df, images, test_size=0.25, random_state=35)
(trainAttrX, testAttrX, trainImagesX, testImagesX) = split

In [29]:
# find the largest house price in the training set and use it to
# scale our house prices to the range [0, 1] (will lead to better
# training and convergence)
maxPrice = trainAttrX["price"].max()
trainY = trainAttrX["price"] / maxPrice
testY = testAttrX["price"] / maxPrice


In [30]:
# import the necessary packages
from keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.layers import Flatten
from keras.layers import Input
from keras.models import Model

def create_mlp(dim, regress=False):
    # define our MLP network
    model = Sequential()
    model.add(Dense(8, input_dim=dim, activation="relu"))
    model.add(Dense(4, activation="relu"))

    # check to see if the regression node should be added
    if regress:
        model.add(Dense(1, activation="linear"))

    # return our model
    return model

def create_cnn(width, height, depth, filters=(16, 32, 64), regress=False):
    # initialize the input shape and channel dimension, assuming
    # TensorFlow/channels-last ordering
    inputShape = (height, width, depth)
    chanDim = -1

    # define the model input
    inputs = Input(shape=inputShape)

    # loop over the number of filters
    for (i, f) in enumerate(filters):
        # if this is the first CONV layer then set the input
        # appropriately
        if i == 0:
            x = inputs

        # CONV => RELU => BN => POOL
        x = Conv2D(f, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)

    # flatten the volume, then FC => RELU => BN => DROPOUT
    x = Flatten()(x)
    x = Dense(16)(x)
    x = Activation("relu")(x)
    x = BatchNormalization(axis=chanDim)(x)
    x = Dropout(0.5)(x)

    # apply another FC layer, this one to match the number of nodes
    # coming out of the MLP
    x = Dense(4)(x)
    x = Activation("relu")(x)

    # check to see if the regression node should be added
    if regress:
        x = Dense(1, activation="linear")(x)

    # construct the CNN
    model = Model(inputs, x)

    # return the CNN
    return model

In [31]:
# create our Convolutional Neural Network and then compile the model
# using mean absolute percentage error as our loss, implying that we
# seek to minimize the absolute percentage difference between our
# price *predictions* and the *actual prices*
model = create_cnn(64, 64, 3, regress=True)
opt = Adam(learning_rate=1e-3, decay=1e-3 / 200)
model.compile(loss="mean_absolute_percentage_error", optimizer=opt)


#train_datagen = ImageDataGenerator(rescale=1./255)
#test_datagen = ImageDataGenerator(rescale=1./255)

In [32]:
# train the model
print("[INFO] training model...")
history = model.fit(trainImagesX, trainY, validation_data=(testImagesX, testY),
	epochs=20, batch_size=8)

[INFO] training model...
Epoch 1/20
34/34 [==============================] - 2s 35ms/step - loss: 905.7103 - val_loss: 75.9342
Epoch 2/20
34/34 [==============================] - 1s 29ms/step - loss: 523.9515 - val_loss: 102.9938
Epoch 3/20
34/34 [==============================] - 1s 26ms/step - loss: 432.4721 - val_loss: 105.9536
Epoch 4/20
34/34 [==============================] - 1s 27ms/step - loss: 367.1100 - val_loss: 88.7677
Epoch 5/20
34/34 [==============================] - 1s 30ms/step - loss: 282.4674 - val_loss: 80.3787
Epoch 6/20
34/34 [==============================] - 1s 26ms/step - loss: 260.6185 - val_loss: 102.5323
Epoch 7/20
34/34 [==============================] - 1s 28ms/step - loss: 284.3802 - val_loss: 193.8989
Epoch 8/20
34/34 [==============================] - 1s 27ms/step - loss: 261.9219 - val_loss: 212.7622
Epoch 9/20
34/34 [==============================] - 1s 26ms/step - loss: 291.3642 - val_loss: 264.6086
Epoch 10/20
34/34 [==============================] 

In [33]:
print("[INFO] predicting house prices...")
preds = model.predict(testImagesX)

# compute the difference between the *predicted* house prices and the
# *actual* house prices, then compute the percentage difference and
# the absolute percentage difference
diff = preds.flatten() - testY
percentDiff = (diff / testY) * 100
absPercentDiff = np.abs(percentDiff)

# compute the mean and standard deviation of the absolute percentage
# difference
mean = np.mean(absPercentDiff)
std = np.std(absPercentDiff)

# finally, show some statistics on our model
from babel.numbers import format_currency
print("[INFO] avg. house price: {}, std house price: {}".format(
	format_currency(df["price"].mean(), 'USD', locale='en_US'),
	format_currency(df["price"].std(), 'USD', locale='en_US')))

[INFO] predicting house prices...
[INFO] avg. house price: $533,388.27, std house price: $493,403.08


In [34]:
from keras.models import Sequential
from keras.layers import Dense
from keras.utils.vis_utils import plot_model
model = Sequential()
model.add(Dense(2, input_dim=1, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')
